In [1]:
import os
import sys
import gc
import pickle
import warnings
import pandas as pd
import numpy as np
import seaborn as sns
from typing import *
from tqdm.notebook import tqdm
from pathlib import Path
from matplotlib import pyplot as plt

pd.set_option('max_columns', 50)
pd.set_option('max_rows', 200)
warnings.simplefilter('ignore')
sns.set()

In [2]:
base_dir = Path().resolve()
sys.path.append(str(base_dir / '../'))

from utils.preprocess import *
from utils.model import *
from utils.train import *
from utils.eval import *
from utils.detectron2helper import *

fail to import apex_C: apex was not installed or installed without --cpp_ext.
fail to import amp_C: apex was not installed or installed without --cpp_ext.
** fvcore version of PathManager will be deprecated soon. **
** Please migrate to the version in iopath repo. **
https://github.com/facebookresearch/iopath 

** fvcore version of PathManager will be deprecated soon. **
** Please migrate to the version in iopath repo. **
https://github.com/facebookresearch/iopath 



HACKING: overriding COCOeval.summarize = vin_summarize...


In [3]:
iou_thr_dfs = pickle.load(open(str(base_dir / 'iou_thr_dfs-11.pkl'), 'rb'))

In [4]:
classes_nms = [
    "Aortic enlargement",
    "Atelectasis",
    "Calcification",
    "Cardiomegaly",
    "Consolidation",
    "ILD",
    "Infiltration",
    "Lung Opacity",
    "Nodule/Mass",
    "Other lesion",
    "Pleural effusion",
    "Pleural thickening",
    "Pneumothorax",
    "Pulmonary fibrosis",
#     "No Finding"
]
classes_dict = {index: class_name  for index, class_name in enumerate(classes_nms)}

In [5]:
N_SPLITS = 5
TEST_META = '../../data/VinBigData/test_meta.csv'

In [6]:
from ensemble_boxes import *


def nms_prediction(pred_df:pd.DataFrame, iou_thr_df: pd.DataFrame, cls_nms: List[str], test_df: pd.DataFrame):
    records = {'image_id': list(), 'PredictionString': list()}

    for i in tqdm(range(len(pred_df))):
        image_id, f_pred_str = pred_df.loc[i]
        image_id_test, height, width = test_df.loc[i]
        assert image_id == image_id_test
        
        boxes, labels, scores = list(), list(), list()

        objs = np.array(f_pred_str.split(' ')).reshape(-1, 6)
        f_boxes = objs[:, 2:].astype(float)
        f_boxes[:, [0, 2]] /= width
        f_boxes[:, [1, 3]] /= height
        f_labels = objs[:, 0].astype(int)
        f_scores = objs[:, 1].astype(float)
        
        for cls_id, cls_nm in enumerate(cls_nms):
            if (f_labels == cls_id).sum() == 0:
                continue
            
            cls_nm_df, iou_thr = iou_thr_df.loc[cls_id, ['cls_nm', 'iou_thr']]
            assert cls_nm == cls_nm_df
            
            c_boxes, c_scores, c_labels = nms_by_class(
                boxes=f_boxes,
                scores=f_scores,
                labels=f_labels,
                cls_id=cls_id,
                iou_thr=iou_thr,
            )

            c_boxes = np.array(c_boxes)
            c_boxes[:, [0, 2]] *= width
            c_boxes[:, [1, 3]] *= height

            boxes += c_boxes.tolist()
            scores += c_scores.tolist()
            labels += c_labels.tolist()

        pred_str = list()
        for label, score, box in zip(labels, scores, boxes):
            pred_str += [str(int(label))] + [str(score)] + np.array(box).astype(str).tolist()

        records['image_id'] += [image_id]
        records['PredictionString'] += [' '.join(pred_str)]

    return pd.DataFrame(records)


def nms_by_class(
    boxes: np.ndarray, scores: np.ndarray, labels: np.ndarray, cls_id: int, iou_thr: float) -> Tuple[List[List[float]], List[float], List[int]]:
    
    mask = (labels == cls_id)
    if mask.sum() == 1:
        return boxes[mask, :], scores[mask], labels[mask]
    else:
        return nms(
            [boxes[mask, :].tolist()],
            [scores[mask].tolist()],
            [labels[mask].tolist()],
            weights=None, 
            iou_thr=iou_thr,
        )


def wbf_prediction(pred_df:pd.DataFrame, iou_thr: float, skip_box_thr: float, cls_nms: List[str], test_df: pd.DataFrame):
    records = {'image_id': list(), 'PredictionString': list()}

    for i in tqdm(range(len(pred_df))):
        image_id, f_pred_str = pred_df.loc[i]
        image_id_test, height, width = test_df.loc[i]
        assert image_id == image_id_test
        
        boxes, labels, scores = list(), list(), list()

        objs = np.array(f_pred_str.split(' ')).reshape(-1, 6)
        f_boxes = objs[:, 2:].astype(float)
        f_boxes[:, [0, 2]] /= width
        f_boxes[:, [1, 3]] /= height
        f_labels = objs[:, 0].astype(int)
        f_scores = objs[:, 1].astype(float)
        
        for cls_id, cls_nm in enumerate(cls_nms):
            if (f_labels == cls_id).sum() == 0:
                continue
            
            c_boxes, c_scores, c_labels = wbf_by_class(
                boxes=f_boxes,
                scores=f_scores,
                labels=f_labels,
                cls_id=cls_id,
                iou_thr=iou_thr,
                skip_box_thr=skip_box_thr,
            )

            c_boxes = np.array(c_boxes)
            c_boxes[:, [0, 2]] *= width
            c_boxes[:, [1, 3]] *= height

            boxes += c_boxes.tolist()
            scores += c_scores.tolist()
            labels += c_labels.tolist()

        pred_str = list()
        for label, score, box in zip(labels, scores, boxes):
            pred_str += [str(int(label))] + [str(np.round(score, 3))] + np.array(box).astype(int).astype(str).tolist()

        records['image_id'] += [image_id]
        records['PredictionString'] += [' '.join(pred_str)]

    return pd.DataFrame(records)


def wbf_by_class(
    boxes: np.ndarray, scores: np.ndarray, labels: np.ndarray, cls_id: int, iou_thr: float, skip_box_thr: float
) -> Tuple[List[List[float]], List[float], List[int]]:
    
    mask = (labels == cls_id)
    if mask.sum() == 1:
        return boxes[mask, :], scores[mask], labels[mask]
    else:
        return weighted_boxes_fusion(
            [boxes[mask, :].tolist()],
            [scores[mask].tolist()],
            [labels[mask].tolist()],
            weights=None, 
            iou_thr=iou_thr,
            skip_box_thr=skip_box_thr
        )


In [7]:
test_df = pd.read_csv(str(base_dir / TEST_META))

In [10]:
model_dirs = list(iou_thr_dfs.keys())
model_dirs

['faster_rcnn_R_50_FPN_3x_mkf_5_1024',
 'faster_rcnn_R_50_FPN_3x_mkf_5_1024_hmean',
 'retinanet_R_101_FPN_3x_mkf_5_1024',
 'retinanet_R_101_FPN_3x_mkf_5_1024_rm',
 'retinanet_R_101_FPN_3x_mkf_5_512',
 'retinanet_R_50_FPN_3x_mkf_5_1024',
 'retinanet_R_50_FPN_3x_mkf_5_512',
 'retinanet_R_50_FPN_3x_nms01_mkf_5_512',
 'cascade_mask_rcnn_R_50_FPN_3x_mkf_5_1024_rm',
 'faster_rcnn_R_101_FPN_3x_mkf_5_1024',
 'faster_rcnn_R_101_FPN_3x_mkf_5_1024_rm']

In [11]:
# apply nms by class

# for model_dir in model_dirs:
#     print(model_dir)
#     iou_thr_df = iou_thr_dfs[model_dir]
#     for fold in range(N_SPLITS):
#         outdir = base_dir / 'detectron2_results' / model_dir / f'fold-{fold + 1}'
#         try:
#             pred_df = pd.read_csv(str(outdir / 'submission.csv'))
#         except FileNotFoundError as e:
#             pred_df = pd.read_csv(str(outdir / 'submmission.csv'))
#         if 
#         pred_df_nms = nms_prediction(pred_df=pred_df, iou_thr_df=iou_thr_df, cls_nms=classes_nms, test_df=test_df)
#         pred_df_nms.to_csv(str(outdir / 'submission_nms.csv'))

In [14]:
# apply wbf by class
IOU_THR = 0.6
SKIP_BOX_THR = 0.05

for model_dir in model_dirs:
    print(model_dir)
    for fold in range(N_SPLITS):
        outdir = base_dir / 'detectron2_results' / model_dir / f'fold-{fold + 1}'
        try:
            pred_df = pd.read_csv(str(outdir / 'submission.csv'))
        except FileNotFoundError as e:
            pred_df = pd.read_csv(str(outdir / 'submmission.csv'))
        pred_df_nms = wbf_prediction(pred_df=pred_df, iou_thr=IOU_THR, skip_box_thr=SKIP_BOX_THR, cls_nms=classes_nms, test_df=test_df)
        pred_df_nms.to_csv(str(outdir / 'submission_wbf.csv'))

faster_rcnn_R_50_FPN_3x_mkf_5_1024


  0%|          | 0/3000 [00:00<?, ?it/s]

  0%|          | 0/3000 [00:00<?, ?it/s]

  0%|          | 0/3000 [00:00<?, ?it/s]

  0%|          | 0/3000 [00:00<?, ?it/s]

...

  0%|          | 0/3000 [00:00<?, ?it/s]

  0%|          | 0/3000 [00:00<?, ?it/s]

  0%|          | 0/3000 [00:00<?, ?it/s]

  0%|          | 0/3000 [00:00<?, ?it/s]

  0%|          | 0/3000 [00:00<?, ?it/s]

In [15]:
submission_2cls = pd.read_csv(str(base_dir / '2-cls test pred.csv'))
submission_2cls.head()

,image_id,target
0,002a34c58c5b758217ed1f584ccbcfe9,0.013326
1,004f33259ee4aef671c2b95d54e4be68,0.037235
2,008bdde2af2462e86fd373a445d0f4cd,0.939700
3,009bc039326338823ca3aa84381f17f1,0.123799
4,00a2145de1886cb9eb88869c85d74080,0.654006


In [169]:
submission_out = pd.read_csv(str(base_dir / 'submission_postprocessed.csv'))
submission_out['is_abnormal'] = (submission_out['PredictionString'] != '14 1 0 0 1 1')
submission_out = submission_out.rename(columns={'PredictionString': 'PredictionString_out'})
submission_out.head()

,image_id,PredictionString_out,is_abnormal
0,002a34c58c5b758217ed1f584ccbcfe9,14 1 0 0 1 1,False
1,004f33259ee4aef671c2b95d54e4be68,11 0.097 1406 302 1701 367 0 0.838 1254 585 15...,True
2,008bdde2af2462e86fd373a445d0f4cd,13 0.009 734 1097 833 1198 11 0.05 1589 439 18...,True
3,009bc039326338823ca3aa84381f17f1,14 1 0 0 1 1,False
4,00a2145de1886cb9eb88869c85d74080,13 0.064 555 1289 809 1406 3 0.854 776 1270 18...,True


In [171]:
for i in range(len(submission_out)):
    objs = np.array(submission_out.loc[i, 'PredictionString_out'].split(' ')).reshape(-1, 6)
    objs = [obj.tolist() for obj in objs if obj[0] != '14']
    submission_out.loc[i, 'PredictionString_out'] = ' '.join(np.array(objs).flatten().tolist())
submission_out.head()

,image_id,PredictionString_out,is_abnormal
0,002a34c58c5b758217ed1f584ccbcfe9,,False
1,004f33259ee4aef671c2b95d54e4be68,11 0.097 1406 302 1701 367 0 0.838 1254 585 15...,True
2,008bdde2af2462e86fd373a445d0f4cd,13 0.009 734 1097 833 1198 11 0.05 1589 439 18...,True
3,009bc039326338823ca3aa84381f17f1,,False
4,00a2145de1886cb9eb88869c85d74080,13 0.064 555 1289 809 1406 3 0.854 776 1270 18...,True


In [172]:
abnormals = list(submission_out.query('is_abnormal == True')['image_id'])

In [45]:
detectron2_dirs = sorted([f for f in os.listdir(str(base_dir / 'detectron2_results')) if f not in ['.ipynb_checkpoints']])
detectron2_dirs

['cascade_mask_rcnn_R_50_FPN_3x_mkf_5_1024_rm',
 'faster_rcnn_R_101_FPN_3x_mkf_5_1024',
 'faster_rcnn_R_101_FPN_3x_mkf_5_1024_rm',
 'faster_rcnn_R_50_FPN_3x_mkf_5_1024',
 'faster_rcnn_R_50_FPN_3x_mkf_5_1024_hmean',
 'retinanet_R_101_FPN_3x_mkf_5_1024',
 'retinanet_R_101_FPN_3x_mkf_5_1024_rm',
 'retinanet_R_101_FPN_3x_mkf_5_512',
 'retinanet_R_50_FPN_3x_mkf_5_1024',
 'retinanet_R_50_FPN_3x_mkf_5_512',
 'retinanet_R_50_FPN_3x_nms01_mkf_5_512']

In [46]:
def check_image_id(df):
    image_ids = df[[c for c in df.columns if 'image_id' in c]].values
    return np.all(list(map(lambda x: len(list(set(x))) == 1, image_ids)))

In [49]:
stack = list()
for model_dir in detectron2_dirs:
    for fold in range(N_SPLITS):
        outdir = base_dir / 'detectron2_results' / model_dir / f'fold-{fold + 1}'
        pred_df = pd.read_csv(str(outdir / 'submission_wbf.csv'))
        pred_df = pred_df.rename(columns={c: f'{c}_{model_dir}_{fold + 1}' for c in pred_df.columns})
        stack += [pred_df]
pred_df_concat = pd.concat(stack, axis=1)
assert check_image_id(pred_df_concat)
image_ids = pred_df_concat[[c for c in pred_df_concat.columns if 'image_id' in c][0]]
pred_df_concat = pred_df_concat[[c for c in pred_df_concat.columns if 'PredictionString' in c]]
pred_df_concat['image_id'] = image_ids
pred_df_concat_detectron2 = pred_df_concat.copy()

In [65]:
pred_df_concat_detectron2.head()

,PredictionString_cascade_mask_rcnn_R_50_FPN_3x_mkf_5_1024_rm_1,PredictionString_cascade_mask_rcnn_R_50_FPN_3x_mkf_5_1024_rm_2,PredictionString_cascade_mask_rcnn_R_50_FPN_3x_mkf_5_1024_rm_3,PredictionString_cascade_mask_rcnn_R_50_FPN_3x_mkf_5_1024_rm_4,PredictionString_cascade_mask_rcnn_R_50_FPN_3x_mkf_5_1024_rm_5,PredictionString_faster_rcnn_R_101_FPN_3x_mkf_5_1024_1,PredictionString_faster_rcnn_R_101_FPN_3x_mkf_5_1024_2,PredictionString_faster_rcnn_R_101_FPN_3x_mkf_5_1024_3,PredictionString_faster_rcnn_R_101_FPN_3x_mkf_5_1024_4,PredictionString_faster_rcnn_R_101_FPN_3x_mkf_5_1024_5,PredictionString_faster_rcnn_R_101_FPN_3x_mkf_5_1024_rm_1,PredictionString_faster_rcnn_R_101_FPN_3x_mkf_5_1024_rm_2,PredictionString_faster_rcnn_R_101_FPN_3x_mkf_5_1024_rm_3,PredictionString_faster_rcnn_R_101_FPN_3x_mkf_5_1024_rm_4,PredictionString_faster_rcnn_R_101_FPN_3x_mkf_5_1024_rm_5,PredictionString_faster_rcnn_R_50_FPN_3x_mkf_5_1024_1,PredictionString_faster_rcnn_R_50_FPN_3x_mkf_5_1024_2,PredictionString_faster_rcnn_R_50_FPN_3x_mkf_5_1024_3,PredictionString_faster_rcnn_R_50_FPN_3x_mkf_5_1024_4,PredictionString_faster_rcnn_R_50_FPN_3x_mkf_5_1024_5,PredictionString_faster_rcnn_R_50_FPN_3x_mkf_5_1024_hmean_1,PredictionString_faster_rcnn_R_50_FPN_3x_mkf_5_1024_hmean_2,PredictionString_faster_rcnn_R_50_FPN_3x_mkf_5_1024_hmean_3,PredictionString_faster_rcnn_R_50_FPN_3x_mkf_5_1024_hmean_4,PredictionString_faster_rcnn_R_50_FPN_3x_mkf_5_1024_hmean_5,...,PredictionString_retinanet_R_101_FPN_3x_mkf_5_1024_rm_2,PredictionString_retinanet_R_101_FPN_3x_mkf_5_1024_rm_3,PredictionString_retinanet_R_101_FPN_3x_mkf_5_1024_rm_4,PredictionString_retinanet_R_101_FPN_3x_mkf_5_1024_rm_5,PredictionString_retinanet_R_101_FPN_3x_mkf_5_512_1,PredictionString_retinanet_R_101_FPN_3x_mkf_5_512_2,PredictionString_retinanet_R_101_FPN_3x_mkf_5_512_3,PredictionString_retinanet_R_101_FPN_3x_mkf_5_512_4,PredictionString_retinanet_R_101_FPN_3x_mkf_5_512_5,PredictionString_retinanet_R_50_FPN_3x_mkf_5_1024_1,PredictionString_retinanet_R_50_FPN_3x_mkf_5_1024_2,PredictionString_retinanet_R_50_FPN_3x_mkf_5_1024_3,PredictionString_retinanet_R_50_FPN_3x_mkf_5_1024_4,PredictionString_retinanet_R_50_FPN_3x_mkf_5_1024_5,PredictionString_retinanet_R_50_FPN_3x_mkf_5_512_1,PredictionString_retinanet_R_50_FPN_3x_mkf_5_512_2,PredictionString_retinanet_R_50_FPN_3x_mkf_5_512_3,PredictionString_retinanet_R_50_FPN_3x_mkf_5_512_4,PredictionString_retinanet_R_50_FPN_3x_mkf_5_512_5,PredictionString_retinanet_R_50_FPN_3x_nms01_mkf_5_512_1,PredictionString_retinanet_R_50_FPN_3x_nms01_mkf_5_512_2,PredictionString_retinanet_R_50_FPN_3x_nms01_mkf_5_512_3,PredictionString_retinanet_R_50_FPN_3x_nms01_mkf_5_512_4,PredictionString_retinanet_R_50_FPN_3x_nms01_mkf_5_512_5,image_id
0,0 0.669 1030 600 1242 917 0 0.186 857 591 1251...,0 0.693 1028 601 1243 915 0 0.218 856 594 1253...,0 0.681 1027 601 1243 908 0 0.211 856 592 1250...,0 0.682 1021 599 1241 916 0 0.234 860 593 1248...,0 0.676 1021 600 1242 926 0 0.256 855 590 1253...,0 0.866 1039 604 1234 913 0 0.1 1149 234 1456 ...,0 0.876 1037 602 1235 920 0 0.152 1148 227 145...,0 0.868 1039 607 1233 908 0 0.101 1144 241 141...,0 0.873 1037 607 1233 912 0 0.096 1109 250 141...,0 0.832 1039 604 1235 913 0 0.088 1151 233 145...,0 0.808 1036 597 1234 923 0 0.073 1036 1087 14...,0 0.837 1038 607 1237 913 0 0.061 1115 246 142...,0 0.804 1033 607 1236 915 0 0.056 1111 250 140...,0 0.819 1035 599 1235 934 0 0.067 1107 1094 14...,0 0.829 1039 598 1236 928 0 0.077 1111 1093 14...,0 0.68 1015 598 1246 917 3 0.036 753 1256 1494...,0 0.808 1024 600 1247 886 3 0.024 758 1297 149...,0 0.634 1017 597 1250 896 3 0.028 750 1293 148...,0 0.638 1006 596 1247 918 3 0.053 756 1279 148...,0 0.603 1021 595 1246 881 3 0.021 757 1275 148...,0 0.62 1031 601 1246 909 3 0.045 765 1293 1488...,0 0.589 1020 605 1241 927 3 0.019 765 1301 149...,0 0.513 1014 598 1248 888 3 0.021 761 1300 148...,0 0.541 1015 607 1244 919 3 0.032 765 1254 148...,0 0.506 1023 602 1247 889 3 0.018 749 1261 149...,

In [60]:
yolo_dirs = ['results01', 'results02']
pred_df_concat_yolo = pd.read_csv(str(base_dir / 'sample_submission.csv'))

for model_dir in yolo_dirs:
    for fold in range(N_SPLITS):
        outdir = base_dir / 'yolov5_results' / model_dir / f'fold-{fold + 1}'
        pred_df = pd.read_csv(str(outdir / 'submission.csv'), index_col=0)
        pred_df = pred_df.rename(columns={'PredictionString': f'PredictionString_{model_dir}_{fold + 1}'})
        
        pred_df_concat_yolo = pd.merge(pred_df_concat_yolo, pred_df, how='left', on='image_id')

In [64]:
pred_df_concat_yolo.head()

,image_id,PredictionString,PredictionString_results01_1,PredictionString_results01_2,PredictionString_results01_3,PredictionString_results01_4,PredictionString_results01_5,PredictionString_results02_1,PredictionString_results02_2,PredictionString_results02_3,PredictionString_results02_4,PredictionString_results02_5
0,002a34c58c5b758217ed1f584ccbcfe9,14 1 0 0 1 1,0 0.207 1167 635 1433 905 3 0.296 815 1223 188...,3 0.428 828 1206 1900 1640 5 0.021 602 1054 92...,3 0.112 824 1206 1919 1637 5 0.042 290 721 918...,0 0.015 1177 638 1447 900 3 0.023 828 1233 190...,0 0.012 1172 625 1433 948 3 0.083 801 1223 191...,0 0.04 1163 646 1438 903 3 0.233 828 1221 1896...,0 0.009 1181 635 1447 903 3 0.464 824 1231 190...,0 0.024 1154 646 1447 923 3 0.096 838 1221 190...,0 0.063 1172 645 1433 898 3 0.014 828 1201 191...,0 0.007 1167 640 1428 888 3 0.625 828 1246 192...
1,004f33259ee4aef671c2b95d54e4be68,14 1 0 0 1 1,0 0.817 1263 585 1531 893 3 0.008 1268 585 152...,0 0.833 1263 582 1526 895 3 0.453 1012 1463 18...,0 0.772 1260 585 1533 890 3 0.012 1012 1437 19...,0 0.825 1258 588 1528 884 0 0.011 1007 582 153...,0 0.816 1263 585 1528 881 3 0.053 1007 1466 19...,0 0.803 1258 579 1528 898 3 0.014 1007 1490 19...,0 0.81 1268 585 1528 887 3 0.011 1022 1514 190...,0 0.74 1263 579 1533 910 3 0.021 1002 1478 191...,0 0.851 1258 585 1528 881 3 0.03 1012 1490 192...,0 0.813 1273 585 1528 898 3 0.127 1007 1490 19...
2,008bdde2af2462e86fd373a445d0f4cd,14 1 0 0 1 1,0 0.786 1440 824 1736 1175 2 0.008 1440 824 17...,0 0.761 1435 821 1737 1184 2 0.017 1620 866 16...,0 0.6 1421 826 1741 1164 3 0.755 1093 1395 193...,0 0.808 1426 832 1732 1192 2 0.08 1611 860 168...,0 0.826 1437 821 1737 1192 2 0.03 1613 854 167...,0 0.812 1426 815 1736 1198 0 0.005 1088 1372 1...,0 0.861 1431 821 1736 1175 0 0.005 1102 1411 1...,0 0.836 1430 815 1746 1215 0 0.007 1098 1417 1...,0 0.825 1431 826 1736 1170 0 0.005 1102 1400 1...,0 0.831 1431 821 1736 1181 0 0.012 1102 1417 1...
3,009bc039326338823ca3aa84381f17f1,14 1 0 0 1 1,0 0.706 996 479 1211 735 3 0.703 648 1063 1563...,0 0.623 996 472 1211 726 0 0.008 780 469 1217 ...,0 0.294 993 472 1215 726 0 0.037 763 469 1215 ...,0 0.547 993 484 1211 735 0 0.124 763 462 1215 ...,0 0.466 996 474 1211 726 3 0.651 662 1063 1553...,0 0.638 989 474 1215 735 0 0.006 778 474 1203 ...,0 0.632 997 479 1215 726 0 0.027 771 479 1230 ...,0 0.59 985 474 1215 730 0 0.006 771 446 1222 9...,0 0.485 985 474 1215 726 0 0.141 771 493 1218 ...,0 0.407 997 469 1215 711 0 0.007 654 1058 1557...
4,00a2145de1886cb9eb88869c85d74080,14 1 0 0 1 1,0 0.765 1118 710 1351 942 2 0.011 1301 771 134...,0 0.677 1122 710 1351 938 3 0.803 771 1293 184...,0 0.667 1115 712 1351 935 2 0.017 1322 776 133...,0 0.775 1128 710 1345 938 2 0.025 1314 754 134...,0 0.684 1124 714 1349 945 3 0.827 765 1302 187...,0 0.762 1118 710 1347 940 3 0.831 784 1279 184...,0 0.79 1126 710 1351 921 2 0.077 1309 761 1347...,0 0.68 1122 705 1351 954 2 0.077 1305 757 1343...,0 0.826 1130 710 1351 931 2 0.013 1293 757 135...,0 0.746 1126 710 1347 926 0 0.006 775 1288 186...


In [173]:
submission_out

,image_id,PredictionString_out,is_abnormal
0,002a34c58c5b758217ed1f584ccbcfe9,,False
1,004f33259ee4aef671c2b95d54e4be68,11 0.097 1406 302 1701 367 0 0.838 1254 585 15...,True
2,008bdde2af2462e86fd373a445d0f4cd,13 0.009 734 1097 833 1198 11 0.05 1589 439 18...,True
3,009bc039326338823ca3aa84381f17f1,,False
4,00a2145de1886cb9eb88869c85d74080,13 0.064 555 1289 809 1406 3 0.854 776 1270 18...,True
...,...,...,...
2995,ff91fb82429a27521bbec8569b041f02,13 0.385 2334 1212 2412 1302 13 0.014 858 1344...,True
2996,ff9fcc4087ed5e941209aa3fa948e364,11 0.075 1296 383 1503 456 7 0.023 819 742 101...,True
2997,ffaa288c8abca300974f043b57d81521,,False
2998,ffc441e0c8b7153844047483a577e7c3,0 0.248 983 646 1230 932,True


In [175]:
pred_df_concat = pd.merge(pred_df_concat_detectron2, pred_df_concat_yolo, how='left', on='image_id')
pred_df_concat = pd.merge(pred_df_concat, submission_out[['image_id', 'PredictionString_out']], how='left', on='image_id')
pred_df_concat.head()

,PredictionString_cascade_mask_rcnn_R_50_FPN_3x_mkf_5_1024_rm_1,PredictionString_cascade_mask_rcnn_R_50_FPN_3x_mkf_5_1024_rm_2,PredictionString_cascade_mask_rcnn_R_50_FPN_3x_mkf_5_1024_rm_3,PredictionString_cascade_mask_rcnn_R_50_FPN_3x_mkf_5_1024_rm_4,PredictionString_cascade_mask_rcnn_R_50_FPN_3x_mkf_5_1024_rm_5,PredictionString_faster_rcnn_R_101_FPN_3x_mkf_5_1024_1,PredictionString_faster_rcnn_R_101_FPN_3x_mkf_5_1024_2,PredictionString_faster_rcnn_R_101_FPN_3x_mkf_5_1024_3,PredictionString_faster_rcnn_R_101_FPN_3x_mkf_5_1024_4,PredictionString_faster_rcnn_R_101_FPN_3x_mkf_5_1024_5,PredictionString_faster_rcnn_R_101_FPN_3x_mkf_5_1024_rm_1,PredictionString_faster_rcnn_R_101_FPN_3x_mkf_5_1024_rm_2,PredictionString_faster_rcnn_R_101_FPN_3x_mkf_5_1024_rm_3,PredictionString_faster_rcnn_R_101_FPN_3x_mkf_5_1024_rm_4,PredictionString_faster_rcnn_R_101_FPN_3x_mkf_5_1024_rm_5,PredictionString_faster_rcnn_R_50_FPN_3x_mkf_5_1024_1,PredictionString_faster_rcnn_R_50_FPN_3x_mkf_5_1024_2,PredictionString_faster_rcnn_R_50_FPN_3x_mkf_5_1024_3,PredictionString_faster_rcnn_R_50_FPN_3x_mkf_5_1024_4,PredictionString_faster_rcnn_R_50_FPN_3x_mkf_5_1024_5,PredictionString_faster_rcnn_R_50_FPN_3x_mkf_5_1024_hmean_1,PredictionString_faster_rcnn_R_50_FPN_3x_mkf_5_1024_hmean_2,PredictionString_faster_rcnn_R_50_FPN_3x_mkf_5_1024_hmean_3,PredictionString_faster_rcnn_R_50_FPN_3x_mkf_5_1024_hmean_4,PredictionString_faster_rcnn_R_50_FPN_3x_mkf_5_1024_hmean_5,...,PredictionString_retinanet_R_50_FPN_3x_mkf_5_1024_4,PredictionString_retinanet_R_50_FPN_3x_mkf_5_1024_5,PredictionString_retinanet_R_50_FPN_3x_mkf_5_512_1,PredictionString_retinanet_R_50_FPN_3x_mkf_5_512_2,PredictionString_retinanet_R_50_FPN_3x_mkf_5_512_3,PredictionString_retinanet_R_50_FPN_3x_mkf_5_512_4,PredictionString_retinanet_R_50_FPN_3x_mkf_5_512_5,PredictionString_retinanet_R_50_FPN_3x_nms01_mkf_5_512_1,PredictionString_retinanet_R_50_FPN_3x_nms01_mkf_5_512_2,PredictionString_retinanet_R_50_FPN_3x_nms01_mkf_5_512_3,PredictionString_retinanet_R_50_FPN_3x_nms01_mkf_5_512_4,PredictionString_retinanet_R_50_FPN_3x_nms01_mkf_5_512_5,image_id,PredictionString,PredictionString_results01_1,PredictionString_results01_2,PredictionString_results01_3,PredictionString_results01_4,PredictionString_results01_5,PredictionString_results02_1,PredictionString_results02_2,PredictionString_results02_3,PredictionString_results02_4,PredictionString_results02_5,PredictionString_out
0,0 0.669 1030 600 1242 917 0 0.186 857 591 1251...,0 0.693 1028 601 1243 915 0 0.218 856 594 1253...,0 0.681 1027 601 1243 908 0 0.211 856 592 1250...,0 0.682 1021 599 1241 916 0 0.234 860 593 1248...,0 0.676 1021 600 1242 926 0 0.256 855 590 1253...,0 0.866 1039 604 1234 913 0 0.1 1149 234 1456 ...,0 0.876 1037 602 1235 920 0 0.152 1148 227 145...,0 0.868 1039 607 1233 908 0 0.101 1144 241 141...,0 0.873 1037 607 1233 912 0 0.096 1109 250 141...,0 0.832 1039 604 1235 913 0 0.088 1151 233 145...,0 0.808 1036 597 1234 923 0 0.073 1036 1087 14...,0 0.837 1038 607 1237 913 0 0.061 1115 246 142...,0 0.804 1033 607 1236 915 0 0.056 1111 250 140...,0 0.819 1035 599 1235 934 0 0.067 1107 1094 14...,0 0.829 1039 598 1236 928 0 0.077 1111 1093 14...,0 0.68 1015 598 1246 917 3 0.036 753 1256 1494...,0 0.808 1024 600 1247 886 3 0.024 758 1297 149...,0 0.634 1017 597 1250 896 3 0.028 750 1293 148...,0 0.638 1006 596 1247 918 3 0.053 756 1279 148...,0 0.603 1021 595 1246 881 3 0.021 757 1275 148...,0 0.62 1031 601 1246 909 3 0.045 765 1293 1488...,0 0.589 1020 605 1241 927 3 0.019 765 1301 149...,0 0.513 1014 598 1248 888 3 0.021 761 1300 148...,0 0.541 1015 607 1244 919 3 0.032 765 1254 148...,0 0.506 1023 602 1247 889 3 0.018 749 1261 149...,...,0 0.39 1012 591 1255 930 0 0.205 875 587 1256 ...,0 0.361 997 595 1252 922 0 0.122 849 584 1257 ...,0 0.486 1031 597 1249 863 0 0.16 906 568 1256 ...,0 0.519 1037 596 1249 865 0 0.201 904 574 1255...,0 0.465 1035 597 1250 860 0 0.217 879 551 1255...,0 0.589 1042 598 1248 860 0 0.234 923 566 1251...,0 0.445 

In [176]:
pred_df_concat_abnormal = pred_df_concat.query(f'image_id in {abnormals}').reset_index(drop=True)
pred_df_concat_abnormal.fillna('', inplace=True)
pred_df_concat_abnormal.head()

,PredictionString_cascade_mask_rcnn_R_50_FPN_3x_mkf_5_1024_rm_1,PredictionString_cascade_mask_rcnn_R_50_FPN_3x_mkf_5_1024_rm_2,PredictionString_cascade_mask_rcnn_R_50_FPN_3x_mkf_5_1024_rm_3,PredictionString_cascade_mask_rcnn_R_50_FPN_3x_mkf_5_1024_rm_4,PredictionString_cascade_mask_rcnn_R_50_FPN_3x_mkf_5_1024_rm_5,PredictionString_faster_rcnn_R_101_FPN_3x_mkf_5_1024_1,PredictionString_faster_rcnn_R_101_FPN_3x_mkf_5_1024_2,PredictionString_faster_rcnn_R_101_FPN_3x_mkf_5_1024_3,PredictionString_faster_rcnn_R_101_FPN_3x_mkf_5_1024_4,PredictionString_faster_rcnn_R_101_FPN_3x_mkf_5_1024_5,PredictionString_faster_rcnn_R_101_FPN_3x_mkf_5_1024_rm_1,PredictionString_faster_rcnn_R_101_FPN_3x_mkf_5_1024_rm_2,PredictionString_faster_rcnn_R_101_FPN_3x_mkf_5_1024_rm_3,PredictionString_faster_rcnn_R_101_FPN_3x_mkf_5_1024_rm_4,PredictionString_faster_rcnn_R_101_FPN_3x_mkf_5_1024_rm_5,PredictionString_faster_rcnn_R_50_FPN_3x_mkf_5_1024_1,PredictionString_faster_rcnn_R_50_FPN_3x_mkf_5_1024_2,PredictionString_faster_rcnn_R_50_FPN_3x_mkf_5_1024_3,PredictionString_faster_rcnn_R_50_FPN_3x_mkf_5_1024_4,PredictionString_faster_rcnn_R_50_FPN_3x_mkf_5_1024_5,PredictionString_faster_rcnn_R_50_FPN_3x_mkf_5_1024_hmean_1,PredictionString_faster_rcnn_R_50_FPN_3x_mkf_5_1024_hmean_2,PredictionString_faster_rcnn_R_50_FPN_3x_mkf_5_1024_hmean_3,PredictionString_faster_rcnn_R_50_FPN_3x_mkf_5_1024_hmean_4,PredictionString_faster_rcnn_R_50_FPN_3x_mkf_5_1024_hmean_5,...,PredictionString_retinanet_R_50_FPN_3x_mkf_5_1024_4,PredictionString_retinanet_R_50_FPN_3x_mkf_5_1024_5,PredictionString_retinanet_R_50_FPN_3x_mkf_5_512_1,PredictionString_retinanet_R_50_FPN_3x_mkf_5_512_2,PredictionString_retinanet_R_50_FPN_3x_mkf_5_512_3,PredictionString_retinanet_R_50_FPN_3x_mkf_5_512_4,PredictionString_retinanet_R_50_FPN_3x_mkf_5_512_5,PredictionString_retinanet_R_50_FPN_3x_nms01_mkf_5_512_1,PredictionString_retinanet_R_50_FPN_3x_nms01_mkf_5_512_2,PredictionString_retinanet_R_50_FPN_3x_nms01_mkf_5_512_3,PredictionString_retinanet_R_50_FPN_3x_nms01_mkf_5_512_4,PredictionString_retinanet_R_50_FPN_3x_nms01_mkf_5_512_5,image_id,PredictionString,PredictionString_results01_1,PredictionString_results01_2,PredictionString_results01_3,PredictionString_results01_4,PredictionString_results01_5,PredictionString_results02_1,PredictionString_results02_2,PredictionString_results02_3,PredictionString_results02_4,PredictionString_results02_5,PredictionString_out
0,0 0.709 1152 887 1435 1206 0 0.139 1218 904 14...,0 0.695 1152 884 1434 1207 0 0.162 1208 906 14...,0 0.708 1148 887 1432 1208 0 0.181 1211 904 14...,0 0.74 1135 887 1434 1209 0 0.152 1208 906 142...,0 0.723 1139 889 1435 1207 0 0.131 1215 906 14...,0 0.85 1155 890 1431 1200 0 0.195 1206 911 142...,0 0.869 1151 890 1430 1206 0 0.247 1247 904 14...,0 0.867 1177 898 1434 1179 0 0.07 1243 913 143...,0 0.856 1155 890 1430 1211 0 0.236 1204 909 14...,0 0.852 1187 897 1428 1161 0 0.229 1064 912 14...,0 0.799 1161 884 1436 1217 0 0.384 1210 902 14...,0 0.764 1165 885 1436 1217 0 0.216 1204 915 14...,0 0.778 1159 889 1439 1218 0 0.533 1210 908 14...,0 0.821 1157 887 1439 1224 0 0.516 1205 903 14...,0 0.789 1155 889 1429 1198 0 0.17 1237 897 140...,0 0.757 1162 884 1429 1165 3 0.945 819 1612 19...,0 0.767 1167 890 1429 1160 3 0.922 825 1607 19...,0 0.781 1166 888 1434 1168 3 0.934 840 1616 19...,0 0.785 1161 885 1435 1174 0 0.062 979 892 144...,0 0.824 1173 892 1435 1169 3 0.88 839 1623 192...,0 0.822 1163 889 1430 1177 2 0.067 1524 1579 1...,0 0.675 1180 894 1435 1168 3 0.843 842 1628 19...,0 0.745 1171 889 1436 1163 3 0.838 829 1613 19...,0 0.677 1165 890 1432 1164 2 0.073 1521 1572 1...,0 0.815 1165 887 1430 1170 2 0.058 1523 1579 1...,...,0 0.481 1181 890 1431 1177 0 0.09 860 1339 190...,0 0.486 1183 888 1434 1176 0 0.117 862 1338 19...,0 0.538 1183 898 1434 1158 0 0.266 1035 883 14...,0 0.506 1191 896 1433 1170 0 0.202 1146 876 14...,0 0.599 1185 898 1436 1160 0 0.264 1040 884 14...,0 0.535 1198 898 1436 1156 0 0.214 1133 859 14...,0 0.576 

In [181]:
stack_cols = [c for c in pred_df_concat_abnormal.columns if c not in ['image_id', 'PredictionString']]
stack_cols

['PredictionString_cascade_mask_rcnn_R_50_FPN_3x_mkf_5_1024_rm_1',
 'PredictionString_cascade_mask_rcnn_R_50_FPN_3x_mkf_5_1024_rm_2',
 'PredictionString_cascade_mask_rcnn_R_50_FPN_3x_mkf_5_1024_rm_3',
 'PredictionString_cascade_mask_rcnn_R_50_FPN_3x_mkf_5_1024_rm_4',
 'PredictionString_cascade_mask_rcnn_R_50_FPN_3x_mkf_5_1024_rm_5',
 'PredictionString_faster_rcnn_R_101_FPN_3x_mkf_5_1024_1',
 'PredictionString_faster_rcnn_R_101_FPN_3x_mkf_5_1024_2',
 'PredictionString_faster_rcnn_R_101_FPN_3x_mkf_5_1024_3',
 'PredictionString_faster_rcnn_R_101_FPN_3x_mkf_5_1024_4',
 'PredictionString_faster_rcnn_R_101_FPN_3x_mkf_5_1024_5',
 'PredictionString_faster_rcnn_R_101_FPN_3x_mkf_5_1024_rm_1',
 'PredictionString_faster_rcnn_R_101_FPN_3x_mkf_5_1024_rm_2',
 'PredictionString_faster_rcnn_R_101_FPN_3x_mkf_5_1024_rm_3',
 'PredictionString_faster_rcnn_R_101_FPN_3x_mkf_5_1024_rm_4',
 'PredictionString_faster_rcnn_R_101_FPN_3x_mkf_5_1024_rm_5',
 'PredictionString_faster_rcnn_R_50_FPN_3x_mkf_5_1024_1',
 '

In [200]:
# ensemble by all class using wbf
IOU_THR = 0.5
SKIP_BOX_THR = 0.1

records = {'image_id': list(), 'PredictionString': list()}

for i in tqdm(range(len(pred_df_concat_abnormal))):
    image_id = pred_df_concat_abnormal.loc[i, 'image_id']
    image_id_test, height, width = test_df.query('image_id == @image_id').values[0]
    assert image_id == image_id_test
    
    boxes, scores, labels = list(), list(), list()
    
    weights = list()
    for pred_str, col in zip(pred_df_concat_abnormal.loc[i, stack_cols], stack_cols):
        if pred_str == '':
            continue
        if col == 'PredictionString_out':
            weights += [30]
        else:
            weights += [1]
            
        objs = np.array(pred_str.split(' ')).reshape(-1, 6)
        m_boxes = objs[:, 2:].astype(float)
        m_boxes[:, [0, 2]] /= width
        m_boxes[:, [1, 3]] /= height
        m_boxes = m_boxes.tolist()
        m_scores = objs[:, 1].astype(float).tolist()
        m_labels = objs[:, 0].astype(int).tolist()

        boxes += [m_boxes]
        scores += [m_scores]
        labels += [m_labels]
    
#     boxes = [np.vstack(boxes).tolist()]
#     scores = [np.hstack(scores).tolist()]
#     labels = [np.hstack(labels).tolist()]
    
    boxes_, scores_, labels_ = weighted_boxes_fusion(boxes, scores, labels, weights=weights, iou_thr=IOU_THR, skip_box_thr=SKIP_BOX_THR)
    boxes_ = np.array(boxes_)
    boxes_[:, [0, 2]] *= width
    boxes_[:, [1, 3]] *= height

    e_pred_str = list()
    for label, score, box in zip(labels_, scores_, boxes_):
        e_pred_str += [str(int(label))] + [str(np.round(score, 3))] + box.astype(int).astype(str).tolist()

    records['image_id'] += [image_id]
    records['PredictionString'] += [' '.join(e_pred_str)]

submission_abnormal = pd.DataFrame(records)

  0%|          | 0/1414 [00:00<?, ?it/s]

In [201]:
submission_abnormal.head()

,image_id,PredictionString
0,1d12b94b7acbeadef7d7700b50aa90d4,3 0.446 835 1576 1925 2017 0 0.436 1171 891 14...
1,6b872791e23742f6c33a08fc24f77365,8 0.134 1753 1965 1832 2073 3 0.106 766 1409 1...
2,645e178a29fb3afd99c82be559afb26e,0 0.786 1153 553 1455 958 3 0.151 786 1341 165...
3,e06a136bef44d96402a353c3073f65e5,3 0.779 865 1224 1969 1838 7 0.248 503 1084 97...
4,93239daad566dbc55904c90e6f74a8f4,0 0.644 1137 779 1416 1090 3 0.271 830 1517 18...


In [202]:
submission = pd.read_csv(str(base_dir / 'sample_submission.csv'))[['image_id']]
submission = pd.merge(submission, submission_abnormal, how='left', on='image_id')
submission.fillna('14 1 0 0 1 1', inplace=True)
submission

,image_id,PredictionString
0,002a34c58c5b758217ed1f584ccbcfe9,14 1 0 0 1 1
1,004f33259ee4aef671c2b95d54e4be68,0 0.772 1248 578 1535 912 3 0.318 992 1394 190...
2,008bdde2af2462e86fd373a445d0f4cd,0 0.786 1417 819 1736 1191 3 0.667 1085 1393 1...
3,009bc039326338823ca3aa84381f17f1,14 1 0 0 1 1
4,00a2145de1886cb9eb88869c85d74080,3 0.818 777 1276 1845 1648 0 0.588 1119 706 13...
...,...,...
2995,ff91fb82429a27521bbec8569b041f02,0 0.749 1614 622 1887 927 3 0.665 1263 1386 23...
2996,ff9fcc4087ed5e941209aa3fa948e364,0 0.762 1129 685 1426 1061 3 0.426 782 1361 16...
2997,ffaa288c8abca300974f043b57d81521,14 1 0 0 1 1
2998,ffc441e0c8b7153844047483a577e7c3,0 0.413 989 633 1233 937 3 0.333 716 1226 1529...


In [203]:
submission_2cls = pd.read_csv(str(base_dir / '2-cls test pred.csv'))
submission_2cls

,image_id,target
0,002a34c58c5b758217ed1f584ccbcfe9,0.013326
1,004f33259ee4aef671c2b95d54e4be68,0.037235
2,008bdde2af2462e86fd373a445d0f4cd,0.939700
3,009bc039326338823ca3aa84381f17f1,0.123799
4,00a2145de1886cb9eb88869c85d74080,0.654006
...,...,...
2995,ff91fb82429a27521bbec8569b041f02,0.936325
2996,ff9fcc4087ed5e941209aa3fa948e364,0.963583
2997,ffaa288c8abca300974f043b57d81521,0.178720
2998,ffc441e0c8b7153844047483a577e7c3,0.225196


In [204]:
submission = pd.merge(submission_2cls, submission, how='left', on='image_id')
submission['class0'] = 1- submission['target']
submission.head()

,image_id,target,PredictionString,class0
0,002a34c58c5b758217ed1f584ccbcfe9,0.013326,14 1 0 0 1 1,0.986674
1,004f33259ee4aef671c2b95d54e4be68,0.037235,0 0.772 1248 578 1535 912 3 0.318 992 1394 190...,0.962765
2,008bdde2af2462e86fd373a445d0f4cd,0.939700,0 0.786 1417 819 1736 1191 3 0.667 1085 1393 1...,0.060300
3,009bc039326338823ca3aa84381f17f1,0.123799,14 1 0 0 1 1,0.876201
4,00a2145de1886cb9eb88869c85d74080,0.654006,3 0.818 777 1276 1845 1648 0 0.588 1119 706 13...,0.345994


In [205]:
def remove_lowscore(df, thr):
    df_ = df.copy()
    for i in range(len(df_)):
        pred_str = df_.loc[i, 'PredictionString']
        objs = np.array(pred_str.split(' ')).reshape(-1, 6)
        objs = np.array([obj for obj in objs if float(obj[1]) > thr])
        objs = objs.flatten()
        df_.loc[i, 'PredictionString'] = ' '.join(objs)
    return df_

In [206]:
submission = remove_lowscore(submission, 0.05)
submission

,image_id,target,PredictionString,class0
0,002a34c58c5b758217ed1f584ccbcfe9,0.013326,14 1 0 0 1 1,0.986674
1,004f33259ee4aef671c2b95d54e4be68,0.037235,0 0.772 1248 578 1535 912 3 0.318 992 1394 190...,0.962765
2,008bdde2af2462e86fd373a445d0f4cd,0.939700,0 0.786 1417 819 1736 1191 3 0.667 1085 1393 1...,0.060300
3,009bc039326338823ca3aa84381f17f1,0.123799,14 1 0 0 1 1,0.876201
4,00a2145de1886cb9eb88869c85d74080,0.654006,3 0.818 777 1276 1845 1648 0 0.588 1119 706 13...,0.345994
...,...,...,...,...
2995,ff91fb82429a27521bbec8569b041f02,0.936325,0 0.749 1614 622 1887 927 3 0.665 1263 1386 23...,0.063675
2996,ff9fcc4087ed5e941209aa3fa948e364,0.963583,0 0.762 1129 685 1426 1061 3 0.426 782 1361 16...,0.036417
2997,ffaa288c8abca300974f043b57d81521,0.178720,14 1 0 0 1 1,0.821280
2998,ffc441e0c8b7153844047483a577e7c3,0.225196,0 0.413 989 633 1233 937 3 0.333 716 1226 1529...,0.774804


In [207]:
NORMAL = '14 1 0 0 1 1'
low_threshold = 0.001
high_threshold = 0.87


c0, c1, c2 = 0, 0, 0
for i in range(len(submission)):
    p0 = submission.loc[i, "class0"]
    if p0 < low_threshold:

        c0 += 1
    elif low_threshold <= p0 and p0 < high_threshold:

        submission.loc[i, "PredictionString"] += f" 14 {p0} 0 0 1 1"
        c1 += 1
    else:

        submission.loc[i, "PredictionString"] = NORMAL
        c2 += 1
        
        
n_normal_after = len(submission.query("PredictionString == @NORMAL"))
print(f"Keep {c0} Add {c1} Replace {c2}")

Keep 0 Add 937 Replace 2063


In [208]:
submission = submission[['image_id', 'PredictionString']]
submission.to_csv(str(base_dir / 'submission.csv'), index=False)
submission

,image_id,PredictionString
0,002a34c58c5b758217ed1f584ccbcfe9,14 1 0 0 1 1
1,004f33259ee4aef671c2b95d54e4be68,14 1 0 0 1 1
2,008bdde2af2462e86fd373a445d0f4cd,0 0.786 1417 819 1736 1191 3 0.667 1085 1393 1...
3,009bc039326338823ca3aa84381f17f1,14 1 0 0 1 1
4,00a2145de1886cb9eb88869c85d74080,3 0.818 777 1276 1845 1648 0 0.588 1119 706 13...
...,...,...
2995,ff91fb82429a27521bbec8569b041f02,0 0.749 1614 622 1887 927 3 0.665 1263 1386 23...
2996,ff9fcc4087ed5e941209aa3fa948e364,0 0.762 1129 685 1426 1061 3 0.426 782 1361 16...
2997,ffaa288c8abca300974f043b57d81521,14 1 0 0 1 1 14 0.8212801720947028 0 0 1 1
2998,ffc441e0c8b7153844047483a577e7c3,0 0.413 989 633 1233 937 3 0.333 716 1226 1529...


In [209]:
submission.loc[2, 'PredictionString']

'0 0.786 1417 819 1736 1191 3 0.667 1085 1393 1940 1799 11 0.105 1688 449 1900 578 0 0.064 1216 791 1766 1306 11 0.058 819 477 1136 752 11 0.056 1605 436 1866 544 14 0.0603003203868866 0 0 1 1'